In [6]:
import sys
import os
import WS
import numpy as np

Analysis code for weather station data 

This code has been used to produce the paper "Detailed analysis of local climate at the CTAO-North site from 20 yr of MAGIC weather station data"   https://dx.doi.org/10.1093/mnras/stae2214


1. Download the data from: https://dx.doi.org/10.5281/zenodo.11279074 
2. Edit and run the python script create_files.py  
3. Edit the main part of the script WS.py 
4. Run different parts of the analysis

In [12]:
# 3. Edit the main part of the script 

resultdir = '../Results'

h5file_long  = 'WS2003-23_long.h5'
# from: https://ftp.cpc.ncep.noaa.gov/cwlinks/norm.daily.nao.cdas.z500.19500101_current.csv
naoi_file = 'norm.daily.nao.cdas.z500.19500101_current.csv'
not_file = 'NOT_2003_2023.h5'    # optionally, if NOT data existing and needed 

is_naoi = False  # set to True if naoi_file is available
is_NOT  = False  # set to True if not_file is available

# Names of variables provided by WS
name_temperature = 'temperature'
name_humidity    = 'humidity'
name_pressure    = 'pressure'
name_ws_current  = 'windSpeedCurrent'
name_ws_gust     = 'windGust'
name_ws_average  = 'windSpeedAverage'
name_wdir_current= 'windDirection'
name_wdir_average= 'windDirectionAverage'    

# Coverages required for the different analysis parts, all coverages in percent
coverage_cut = 50.
coverage_cut_for_daily_samples = 10
coverage_cut_for_monthly_samples = 80
day_coverage_for_samples = 85  
day_coverage_for_diurnal = 95  

# Wind binning of the Reinhardt station, see Eq. (1) of https://dx.doi.org/10.1093/mnras/stae2214
wind_binning = np.concatenate((np.arange(0.,104.,4.)-0.5,105.2 + 4*(np.arange(0,10)-0.5) + 0.3*(np.arange(0,10)-0.5)**2),axis=None)

# Dates of the weather data
WS_start      = '2003-01-01 00:00:01'   # Start of first data entry 
WS_relocation = '2004-03-01 00:00:01'   # Relocation of the station, see Section 2.1 of https://dx.doi.org/10.1093/mnras/stae2214
new_WS        = '2007-03-20 00:00:01'   # Purchase of new station MWS 5MW and upgrade 
new_model     = '2017-04-10 00:00:01'   # Upgrade to a new MWS 55V 
old_model     = '2023-01-16 00:00:01'   # Exchange back to repaired MWS 5MW
NOT_end_of_5min = '2007-02-02 12:00:00'  # Optional, if NOT data available 
NOT_end_of_data = '2019-12-31 23:59:59'  # Optional, if NOT data available

In [8]:
from setup_helper import SetUp
SetUp()

[array([0.12156863, 0.46666667, 0.70588235, 1.        ]), array([1.        , 0.49803922, 0.05490196, 1.        ]), array([0.17254902, 0.62745098, 0.17254902, 1.        ]), array([0.83921569, 0.15294118, 0.15686275, 1.        ]), array([0.58039216, 0.40392157, 0.74117647, 1.        ]), array([0.54901961, 0.3372549 , 0.29411765, 1.        ]), array([0.89019608, 0.46666667, 0.76078431, 1.        ]), array([0.49803922, 0.49803922, 0.49803922, 1.        ]), array([0.7372549 , 0.74117647, 0.13333333, 1.        ]), array([0.09019608, 0.74509804, 0.81176471, 1.        ]), (0.8627450980392157, 0.0784313725490196, 0.23529411764705882), (0.29411764705882354, 0.0, 0.5098039215686274)]


In [13]:
# Likelihood initialization, do not change if necessary 
name_mu              = ['A', 'B', 'C', 'phi_mu',          'sigma0' ]
name_mu_nob          = ['A', 'C',      'phi_mu',          'sigma0' ]
name_mu_dphim        = ['A', 'B', 'C', 'phi_mu', 'dphim', 'sigma0' ]
name_mu_dCm          = ['A', 'B', 'C', 'phi_mu', 'dCm',   'sigma0' ]
name_mu_offset       = ['A', 'B', 'C', 'phi_mu',          'sigma0', 'offset' ]
name_mu2             = ['A', 'B', 'C', 'phi_mu', 'D', 'phi_mu2','sigma0' ]
name_mu2_sig2        = ['A', 'B', 'C', 'phi_mu', 'D', 'phi_mu2','sigma0', 'E', 'phi_sig' ]    
name_mu2_offset      = ['A', 'B', 'C', 'phi_mu', 'D', 'phi_mu2','sigma0' ,                'offset' ]
name_mu2_sig2_offset = ['A', 'B', 'C', 'phi_mu', 'D', 'phi_mu2','sigma0', 'E', 'phi_sig', 'offset' ]    
bounds_mu              = ((0, None), (None, None), (0, None), (0, None), (0.1,None))
bounds_mu_nob          = ((0, None),               (0, None), (0, None), (0.1,None))
bounds_mu_dphim        = ((0, None), (None, None), (0, None), (0, None), (None,None),             (0.1,None))
bounds_mu_dCm          = ((0, None), (None, None), (0, None), (0, None), (None,None),             (0.1,None))
bounds_mu_offset       = ((0, None), (None, None), (0, None), (0, None), (0.1,None), (None, None))
bounds_mu2             = ((0, None), (None, None), (None, None), (0, None), (0, None), (0, None), (0.1,None))
bounds_mu2_sig2        = ((0, None), (None, None), (None, None), (0, None), (0, None), (0, None), (0.1,None), (-10.,None), (0, 12.))
bounds_mu2_offset      = ((0, None), (None, None), (None, None), (0, None), (0, None), (0, None), (0.1,None),                        (None,None))
bounds_mu2_sig2_offset = ((0, None), (None, None), (None, None), (0, None), (0, None), (0, None), (0.1,None), (-10.,None), (0, 12.), (None,None))

method='L-BFGS-B'
tol=1e-9

In [16]:
WS.plot_huracans()

NameError: name 'dff' is not defined